In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
pd.set_option('display.max_columns', 30)
import re

In [2]:
pwd!

'/Users/suneethi/Documents/nssdatasc/dq4_nashville_metro/dq4-metro-beetles'

In [3]:
metro_trash=pd.read_excel('data/Trash Hauler Report - Nov 2017 - Nov 2019 Final.xlsx', skiprows=1)
metro=pd.read_csv('data/hubNashville__311__Service_Requests.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
metro=metro.drop(columns=['Request Origin', 'System of Origin ID', 'Contact Type', 
                              'Preferred Language', 'State Issue'])

In [5]:
metro.head()

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,108110,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Request an Additional Cart,10/10/18 16:14,3/5/19 21:34,NaN,False,413 Freda Villa,MADISON,8.0,37115,36.243960,-86.727210,"(36.24396, -86.72721)"
1,122078,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,Trash in Recycling Cart,12/7/18 14:50,12/7/18 14:50,NaN,True,"4906 Dakota Ave, Nashville, TN 37209, United S...",NASHVILLE,24.0,37209,36.147680,-86.846440,"(36.14768, -86.84644)"
2,113730,Closed,"Streets, Roads & Sidewalks",Request New Sign,NaN,11/1/18 10:00,11/6/18 11:15,NaN,True,"873 Granada Ave, Nashville, TN 37206, United S...",NASHVILLE,5.0,37206,36.189230,-86.756430,"(36.18923, -86.75643)"
3,124624,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,Trash in Recycling Cart,12/19/18 17:47,12/19/18 17:47,NaN,False,4304 Lindawood Dr,NASHVILLE,34.0,37215,36.097154,-86.831878,"(36.0971543, -86.8318783)"
4,108241,Closed,"Streets, Roads & Sidewalks",Street Lighting,Report Outage,10/10/18 14:49,10/24/18 16:27,NaN,True,"336 Blackman Rd, Nashville, TN 37211, United S...",NASHVILLE,26.0,37211,36.074580,-86.746960,"(36.07458, -86.74696)"


In [6]:
metro=metro.rename(columns={'Request #': 'Request_id', 'Request Type':'Request_type',
                                'Subrequest Type': 'Subrequest_type', 
                                'Additional Subrequest Type': 'Additional_subrequest',
                               'Date / Time Opened' :'dt_opened', 'Date / Time Closed':'dt_closed',
                               'Council District': 'Council_dist', 'ZIP':'Zip', 'Latitude':'Lat', 
                               'Longitude':'Long', 'Mapped Location': 'Location'})

In [7]:
metro_misd=metro[(metro['Subrequest_type']=='Missed Pickup - Recycle Cart Service') | (metro['Subrequest_type']==
                                                                                'Missed Pickup - Trash Cart Service')
         | (metro['Subrequest_type']=='Missed Pickup - Cart Service') | (metro['Subrequest_type']=='Missed Pickup - Dumpster Service')]

In [8]:
metro_trash.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
0,25270,2017-11-01 07:09:00,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066


keywords = ['miss', 'missed']
keywords_required = ['miss', 'missed']
#keywords_not_required = ['Damage to Property']

def check_keywords(df, keywords):
    contains_keywords = pd.DataFrame()
    for keyword in keywords:
        contains_keyword = pd.DataFrame()
        for column in df:
            if df[column].dtype not in ['float64', 'int64']:
                contains_keyword[column] = df[column].str.lower().str.contains(keyword.lower())
        contains_keywords[keyword] = contains_keyword.any(axis=1)
    return contains_keywords.astype('int64')

keyword_scores = check_keywords(metro_trash, keywords)
keyword_required_score = keyword_scores[keywords_required].sum(axis=1)
#keyword_not_required_score = keyword_scores[keywords_not_required].sum(axis=1)

metro_keyword_scores = pd.concat([metro_trash['Request'], keyword_required_score], axis=1)
metro_keyword_scores.columns = ['Request', 'keyword_required_score']
metro_keywd = pd.merge(metro_trash, metro_keyword_scores)


In [9]:
metro_trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 11 columns):
Request Number      20226 non-null int64
Date Opened         20226 non-null datetime64[ns]
Request             20226 non-null object
Description         20197 non-null object
Incident Address    20217 non-null object
Zip Code            20151 non-null float64
Trash Hauler        19325 non-null object
Trash Route         19279 non-null object
Council District    20177 non-null float64
State Plan X        20198 non-null float64
State Plan Y        20198 non-null float64
dtypes: datetime64[ns](1), float64(4), int64(1), object(5)
memory usage: 1.7+ MB


In [10]:
metro_trash=metro_trash.rename(columns={'Request Number': 'Request_id', 'Date Opened' :'dt_opened', 
                                            'Incident Address': 'Address', 'Zip Code': 'Zip', 'Trash Hauler': 'Hauler'
                                           ,'Council District': 'Council_dist' })

In [11]:
metro_trash.shape

(20226, 11)

In [12]:
metro_trash.head()

,Request_id,dt_opened,Request,Description,Address,Zip,Hauler,Trash Route,Council_dist,State Plan X,State Plan Y
0,25270,2017-11-01 07:09:00,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066


In [13]:
metro_trash=pd.merge(metro_trash, metro, on='Request_id', how='inner')

In [14]:
metro_trash.head()

,Request_id,dt_opened_x,Request,Description,Address_x,Zip_x,Hauler,Trash Route,Council_dist_x,State Plan X,State Plan Y,Status,Request_type,Subrequest_type,Additional_subrequest,dt_opened_y,dt_closed,Parent Request,Closed When Created,Address_y,City,Council_dist_y,Zip_y,Lat,Long,Location
0,25270,2017-11-01 07:09:00,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Backdoor,11/1/17 7:09,11/2/17 16:02,NaN,True,3817 Crouch Dr,NASHVILLE,2.0,37207,36.21730,-86.81540,"(36.2173, -86.8154)"
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Curbside/Alley Missed Pickup,11/1/17 7:42,11/2/17 16:02,NaN,True,4028 Clarksville Pike,NASHVILLE,1.0,37218,36.21348,-86.83811,"(36.21348, -86.83811)"
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Curbside/Alley Missed Pickup,11/1/17 7:53,11/2/17 16:00,NaN,True,6528 Thunderbird Dr,NASHVILLE,20.0,37209,36.14293,-86.88557,"(36.14293, -86.88557)"
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Curbside/Alley Missed Pickup,11/1/17 8:28,11/7/17 4:03,NaN,True,2603 old matthews rd,NASHVILLE,2.0,37207,36.21266,-86.78918,"(36.21266, -86.78918)"
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,Trash - Curbside/Alley Missed Pickup,11/1/17 8:55,11/2/17 16:00,NaN,True,604 croley dr,NASHVILLE,20.0,37209,36.15487,-86.87500,"(36.15487, -86.875)"


In [15]:
metro_trash.shape

(20226, 26)

In [16]:
metro_trash=metro_trash.drop(columns=['Additional_subrequest', 'Council_dist_x', 'Zip_x', 'dt_opened_y', 'Address_y'])

In [17]:
metro_trash=metro_trash.rename(columns={'dt_opened_x': 'dt_opened', 'Request ': 'Request', 'Address_x': 'Address',
                                            'Zip_y': 'Zip', 'Trash Route': 'Route'})

In [18]:
metro_trash.head()

,Request_id,dt_opened,Request,Description,Address,Hauler,Route,State Plan X,State Plan Y,Status,Request_type,Subrequest_type,dt_closed,Parent Request,Closed When Created,City,Council_dist_y,Zip,Lat,Long,Location
0,25270,2017-11-01 07:09:00,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,RED RIVER,3205,1.727970e+06,686779.478089,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:02,NaN,True,NASHVILLE,2.0,37207,36.21730,-86.81540,"(36.2173, -86.8154)"
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,RED RIVER,4202,1.721259e+06,685444.799565,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:02,NaN,True,NASHVILLE,1.0,37218,36.21348,-86.83811,"(36.21348, -86.83811)"
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,RED RIVER,4205,1.707027e+06,659887.471571,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,20.0,37209,36.14293,-86.88557,"(36.14293, -86.88557)"
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,WASTE IND,2206,1.735692e+06,685027.245923,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/7/17 4:03,NaN,True,NASHVILLE,2.0,37207,36.21266,-86.78918,"(36.21266, -86.78918)"
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,RED RIVER,4203,1.710186e+06,664205.101066,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,20.0,37209,36.15487,-86.87500,"(36.15487, -86.875)"


In [19]:
metro_trash.Request.value_counts

<bound method IndexOpsMixin.value_counts of 0                            Trash - Backdoor
1        Trash - Curbside/Alley Missed Pickup
2        Trash - Curbside/Alley Missed Pickup
3        Trash - Curbside/Alley Missed Pickup
4        Trash - Curbside/Alley Missed Pickup
                         ...                 
20221    Trash - Curbside/Alley Missed Pickup
20222    Trash - Curbside/Alley Missed Pickup
20223    Trash - Curbside/Alley Missed Pickup
20224    Trash - Curbside/Alley Missed Pickup
20225    Trash - Curbside/Alley Missed Pickup
Name: Request, Length: 20226, dtype: object>

In [20]:
metro_trash.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20226 entries, 0 to 20225
Data columns (total 21 columns):
Request_id             20226 non-null int64
dt_opened              20226 non-null datetime64[ns]
Request                20226 non-null object
Description            20197 non-null object
Address                20217 non-null object
Hauler                 19325 non-null object
Route                  19279 non-null object
State Plan X           20198 non-null float64
State Plan Y           20198 non-null float64
Status                 20226 non-null object
Request_type           20226 non-null object
Subrequest_type        20226 non-null object
dt_closed              20131 non-null object
Parent Request         116 non-null float64
Closed When Created    20226 non-null bool
City                   20178 non-null object
Council_dist_y         20177 non-null float64
Zip                    20151 non-null object
Lat                    20201 non-null float64
Long                   20201

In [21]:
metro_trash['Request']=metro_trash['Request'].astype(str)
metro_trash['Description']=metro_trash['Description'].astype(str)
metro_trash['Req_desc']=metro_trash['Request']+metro_trash['Description']
metro_trash=metro_trash[metro_trash['Req_desc'].str.contains('[Mm]iss', regex=True, na=False)]



In [22]:
metro_trash['Address']=metro_trash['Address'].astype(str)
metro_trash['Address']=metro_trash['Address'].str.lower()
metro_trash['new_add']=metro_trash['Address'].str.replace(r'(nashville)', r',nashville')
metro_trash['new_add']=metro_trash['new_add'].replace(regex=True, to_replace=r'\,.+', value=r'').str.strip()


In [23]:
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'drive', r'dr')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(road)',r'rd')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(boulevard)',r'blvd')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(court)',r'ct')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(trace)',r'trce')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(avenue)',r'ave')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(circle)',r'cir')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(street)',r'st')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(place)',r'pl')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(pike|pke)',r'pk')
metro_trash['new_add']=metro_trash['new_add'].str.replace(r'(lane)',r'ln')
metro_trash.head()

,Request_id,dt_opened,Request,Description,Address,Hauler,Route,State Plan X,State Plan Y,Status,Request_type,Subrequest_type,dt_closed,Parent Request,Closed When Created,City,Council_dist_y,Zip,Lat,Long,Location,Req_desc,new_add
1,25274,2017-11-01 07:42:00,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 clarksville pike,RED RIVER,4202,1.721259e+06,685444.799565,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:02,NaN,True,NASHVILLE,1.0,37218,36.21348,-86.83811,"(36.21348, -86.83811)",Trash - Curbside/Alley Missed PickupCurb/Trash...,4028 clarksville pk
2,25276,2017-11-01 07:53:00,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 thunderbird dr,RED RIVER,4205,1.707027e+06,659887.471571,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,20.0,37209,36.14293,-86.88557,"(36.14293, -86.88557)",Trash - Curbside/Alley Missed PickupCurb/trash...,6528 thunderbird dr
3,25307,2017-11-01 08:28:00,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,WASTE IND,2206,1.735692e+06,685027.245923,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/7/17 4:03,NaN,True,NASHVILLE,2.0,37207,36.21266,-86.78918,"(36.21266, -86.78918)",Trash - Curbside/Alley Missed Pickupmissed,2603 old matthews rd
4,25312,2017-11-01 08:55:00,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,RED RIVER,4203,1.710186e+06,664205.101066,Closed,"Trash, Recycling & Litter",Missed Pickup - Trash Cart Service,11/2/17 16:00,NaN,True,NASHVILLE,20.0,37209,36.15487,-86.87500,"(36.15487, -86.875)",Trash - Curbside/Alley Missed PickupMissed the...,604 croley dr
7,25327,2017-11-01 09:32:00,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 jo johnston ave,METRO,9208,1.731459e+06,666013.601229,Closed,"Trash, Recycling & Litter",File a Trash or Recycling Complaint,11/2/17 12:03,NaN,True,NASHVILLE,21.0,37203,36.16034,-86.80299,"(36.16034, -86.80299)","Trash Collection ComplaintTrash out on time, m...",1816 jo johnston ave


In [24]:
metro_trash['Route']=metro_trash['Route'].astype(str)
metro_trash['Rte']=metro_trash['Route'].str.get(1)